### Production

In [2]:
import mlflow
import pyarrow.feather as feather


In [3]:
X_train = feather.read_feather("./results/X_train.ftr")
X_test = feather.read_feather("./results/X_test.ftr")
y_train = feather.read_feather("./results/y_train.ftr").to_numpy().ravel()
y_test = feather.read_feather("./results/y_test.ftr").to_numpy().ravel()

In [11]:
X_test.iloc[0]

Customer Type               0.476040
Age                         0.817887
Type of Travel              0.663378
Class                      -1.077065
Flight Distance            -1.041795
Inflight wifi service       1.703853
Ease of Online booking      0.172143
Gate location               0.797831
Food and drink             -0.161739
Online boarding             0.544692
Seat comfort               -0.340303
Inflight entertainment      1.227138
On-board service            1.259170
Leg room service            1.250974
Baggage handling            1.161716
Checkin service            -1.035348
Inflight service            1.144063
Cleanliness                 1.298998
Arrival Delay in Minutes    1.948498
Name: 0, dtype: float64

In [12]:
logged_model = 'runs:/8a0ad36647ea44cf97c2c40eb4ca3bf1/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(X_test))



array([1, 1, 0, ..., 0, 1, 0])

In [ ]:
# mlflow models serve -m ./mlruns/0/8a0ad36647ea44cf97c2c40eb4ca3bf1/artifacts/model -p 1234
# curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["Customer Type", "Age", "Type of Travel", "Class", "Flight Distance", "Inflight wifi service", "Ease of Online booking", "Gate location", "Food and drink", "Online boarding", "Seat comfort", "Inflight entertainment", "On-board service", "Leg room service", "Baggage handling", "Checkin service", "Inflight service", "Cleanliness", "Arrival Delay in Minutes"],"data":[[0.47603968,  0.81788702,  0.66337845, -1.07706454, -1.04179459, 1.70385282, 0.17214266,  0.79783121, -0.1617392 , 0.54469211, -0.34030327, 1.22713822, 1.25917014, 1.25097391, 1.16171635, -1.03534761, 1.14406253, 1.29899809, 1.94849759]]}' http://127.0.0.1:1234/invocations